In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [2]:
base_url = 'https://archiveofourown.org/tags/Haruno%20Sakura*s*Uchiha%20Obito/works'

response = requests.get(base_url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html')
    print('success')
else:
    print(f'Failed: {response.status_code}')

success


In [3]:
def extract_data(soup):
    # creating dataframe columns
    # df = pd.DataFrame(columns=['Title', 'Author', 'Fandom', 'Relationship', 'Date',
    #                            'Word Number', 'Chapters','Kudos'])

    
    titles = []
    authors = []
    fandoms = []
    relationships = []
    dates = []
    word_nums = []
    chapters = []
    kudos = []

    bookmarks = []

    articles = soup.find_all('li',role="article")

    for article in articles:
        # finding the title
        heading = article.find('h4', class_='heading').find('a') if article.find('h4', class_='heading') else None
        if heading:
            title = heading.text
            if title:
                titles.append(title)
        
        # finding author
        heading1 = article.find('h4', class_='heading')
        if heading1:
            author = heading1.find('a', rel = 'author')
            if author:
                authors.append(author.text)
            else:
                authors.append('Anonymous')

        # find the fandom
        heading2 = article.find('h5', class_ = 'fandoms heading')
        if heading2:
            span = heading2.find(class_ = 'tag')
            if span:
                fandoms.append(span.text)
        
        # finding relatioships
        heading3 = article.find('ul', class_='tags commas')
        if heading3:
            head = heading3.find('li', class_ = 'relationships')
            if head:
                rela = head.find('a', class_ = 'tag')
                if rela:
                    relationships.append(rela.text)
            else:
                relationships.append('None')
            

    
        # finding dates and filtering it
        heading4 = article.find('ul', class_ = 'recent module group')
        if heading4:
            bookmark = heading4.find(class_ = 'datetime')
            if bookmark:
                bookmarks.append(bookmark.text)

        date = article.find(class_ = 'datetime')
        if date:
            dates.append(date.text)    

        #filterign
        dates =[item for item in dates if item not in bookmarks] 

        # word numbers
        word_num = article.find('dd', class_ = 'words')
        if word_num:
            word_nums.append(word_num.text)

        # chapters
        heading5 = article.find('dl', class_= 'stats')
        if heading5:
            chapter = heading5.find('dd', class_ ='chapters')
            if chapter:
                chapters.append(chapter.text)
        
        # kudos
        kudo = article.find('dd', class_='kudos')
        if kudo:
            kudos.append(kudo.text)
    
    # creating dataframe
    df = pd.DataFrame(list(zip(titles, authors, fandoms,relationships, dates,
                               word_nums, chapters,kudos)), columns=['Title', 'Author', 'Fandom', 'Relationship', 'Date',
                            'Word Number', 'Chapters','Kudos'])
    
    return df
                



In [4]:
def pagination(base_url):
    all_data = []
    page_num = 1

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    while True:
        url = f'{base_url}?page={page_num}'

        print(f'scrapping page num = {page_num} in url: {url}')
        
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html')

        

        page_data = extract_data(soup)

        if page_data.empty:
            break

        all_data.append(page_data)
        page_num += 1
        time.sleep(1)

    if all_data:
        return pd.concat(all_data, ignore_index=True)


In [5]:
ao3_data = pagination(base_url=base_url)

scrapping page num = 1 in url: https://archiveofourown.org/tags/Haruno%20Sakura*s*Uchiha%20Obito/works?page=1
scrapping page num = 2 in url: https://archiveofourown.org/tags/Haruno%20Sakura*s*Uchiha%20Obito/works?page=2
scrapping page num = 3 in url: https://archiveofourown.org/tags/Haruno%20Sakura*s*Uchiha%20Obito/works?page=3
scrapping page num = 4 in url: https://archiveofourown.org/tags/Haruno%20Sakura*s*Uchiha%20Obito/works?page=4
scrapping page num = 5 in url: https://archiveofourown.org/tags/Haruno%20Sakura*s*Uchiha%20Obito/works?page=5
scrapping page num = 6 in url: https://archiveofourown.org/tags/Haruno%20Sakura*s*Uchiha%20Obito/works?page=6
scrapping page num = 7 in url: https://archiveofourown.org/tags/Haruno%20Sakura*s*Uchiha%20Obito/works?page=7
scrapping page num = 8 in url: https://archiveofourown.org/tags/Haruno%20Sakura*s*Uchiha%20Obito/works?page=8


In [6]:
ao3 =pd.DataFrame(ao3_data)
ao3

,Title,Author,Fandom,Relationship,Date,Word Number,Chapters,Kudos
0,Bunnies,PearlySaki,Naruto (Anime & Manga),Haruno Sakura/Hatake Kakashi,15 Sep 2024,"3,149",1/?,2
1,KONOHA NO SAKUYA BIME,Grace82,火影忍者（动漫和漫画）,Haruno Sakura/Hatake Kakashi,15 Sep 2024,"335,958",47/?,476
2,I have 9 soulmates,Grace82,Naruto (Anime & Manga),Haruno Sakura/Hatake Kakashi,14 Sep 2024,"44,119",7/?,162
3,jimsonweed,Sugarcream,Naruto (Anime & Manga),Haruno Sakura/Hatake Kakashi,11 Sep 2024,"10,923",2/2,1
4,Rainha do harém - English Version,CherryYui,Naruto (Anime & Manga),Haruno Sakura/Uchiha Sasuke,06 Sep 2024,"112,414",19/?,278
...,...,...,...,...,...,...,...,...
116,On the Edge of Oblivion,Amesaya,Naruto,Haruno Sakura/Uchiha Obito,08 Aug 2016,"4,907",1/1,204
117,Just a little,orphan_account,Naruto,Haruno Sakura/Uchiha Obito,26 Jun 2016,"1,038",1/1,95
118,"Uchiha Sandwich, With a Side of Madara",greentea815,Naruto,Haruno Sakura/Uchiha Sasuke,23 Apr 2016,"3,190",1/1,674
119,Sequential,PrinceEverhard,Naruto,Haruno Sakura/Uchiha Obito,30 Jan 2013,"1,001",1/1,118


In [8]:
ao3.to_csv('AO3_data.csv', index=False)
print('successfully save the file')

successfully save the file
